# Imports & setup

In [1]:
# Import selenium
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [2]:
# Import undetected chromedriver
import undetected_chromedriver as uc

In [3]:
# Import time & pandas
import time
import pandas as pd

In [4]:
# Set Chrome Options, specifically to allow location tracking on PrizePicks
capabilities = DesiredCapabilities().CHROME

chrome_options = uc.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-popup-blocking")

prefs = {
    'profile.default_content_setting_values':
    {
        'notifications': 1,
        'geolocation': 1
    },

    'profile.managed_default_content_settings':
    {
        'geolocation': 1
    },
}

chrome_options.add_experimental_option('prefs', prefs)
capabilities.update(chrome_options.to_capabilities())

In [5]:
# Open browser
driver = uc.Chrome(options = chrome_options)

In [6]:
# Visit PrizePicks Website
driver.get("https://app.prizepicks.com/")
time.sleep(5)

In [7]:
# Close initial pop-up
WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "close")))
time.sleep(5)
driver.find_element(By.XPATH, "/html/body/div[3]/div[3]/div/div/button").click()
time.sleep(5)

In [8]:
# Navigate to COD tab
driver.find_element(By.XPATH, "//div[@class='name'][normalize-space()='COD']").click()
time.sleep(5)

In [9]:
# Initilize an empty list to hold the player props
player_props = []

In [10]:
# Get the stat container and various stats 
stat_container = WebDriverWait(driver,
                               1).until(EC.visibility_of_element_located((By.CLASS_NAME, "stat-container")))
categories = driver.find_element(By.CSS_SELECTOR, ".stat-container").text.split('\n')

In [11]:
# Remove Maps 1 - 3 Kills & Maps 1 - 3 Kills (Combo)
if "MAPS 1-3 Kills (Combo)" in categories:
    categories.remove("MAPS 1-3 Kills (Combo)")
if "MAPS 1-3 Kills" in categories:
    categories.remove("MAPS 1-3 Kills")
categories

['MAP 1 Kills', 'MAP 2 Kills', 'MAP 3 Kills']

In [12]:
# Get player projections

# Print the beginning of the logging.
print("-----------------------------")

print("Beginning Data Retrieval     ")
print("-----------------------------")
print("")

# Loop through categories
for category in categories:
    driver.find_element(By.XPATH, f"//div[text()='{category}']").click()
    time.sleep(5)

    # Test print statement
    print(f"{category}")
    
    # Get list of all projections for current category
    projectionsPP = driver.find_elements(By.ID, "test-projection-li")

    # Loop through current list of projections
    for i in range(len(projectionsPP)):

        # Get line info for each player prop
        player = projectionsPP[i].find_element(By.ID, "test-player-name").text
        team_abbr = projectionsPP[i].find_element(By.ID, "test-team-position").text.split(" - ")[0]
        player_line = float(driver.find_element(
            By.XPATH, 
            '/html/body/div[1]/div/div[3]/div[1]/div/main/div/div/div[1]/div[3]/ul/li[' + str(i + 1) + ']/div[3]/div/div/div/div[1]'
        ).text)

        # Append prop to our list
        player_props.append({
            "player": player, 
            "team_abbr": team_abbr, 
            "prop": category.split(" ")[1], 
            "line": player_line
        })

        # Test print statement
        print(f"{player} {team_abbr} | {player_line}")

    # Print Formatting
    print("")
    
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

-----------------------------
Beginning Data Retrieval     
-----------------------------

MAP 1 Kills
Huke SEA | 23.5
04 SEA | 24.5
Breszy SEA | 22.0
Abuzah SEA | 22.0
ReeaL MIA | 24.0
Vikul MIA | 22.5
MettalZ MIA | 22.5
Lucky MIA | 21.0
Lynz MIN | 25.0
Standy MIN | 23.5
Gunless MIN | 22.0
Accuracy MIN | 21.5
CleanX TOR | 26.0
Envoy TOR | 23.0
Scrap TOR | 25.5
Insight TOR | 21.5
JoeDeceives LAT | 25.5
Kremp LAT | 24.5
Ghosty LAT | 25.0
Nastie LAT | 23.5
Snoopy BOS | 26.0
Pentagrxm BOS | 23.0
Priestahh BOS | 23.0
Seany BOS | 23.5

MAP 2 Kills
Huke SEA | 7.0
04 SEA | 7.5
Breszy SEA | 6.5
Abuzah SEA | 6.5
ReeaL MIA | 6.0
Vikul MIA | 6.5
MettalZ MIA | 6.0
Lucky MIA | 7.0
Lynz MIN | 7.0
Standy MIN | 6.0
Gunless MIN | 6.0
Accuracy MIN | 6.5
CleanX TOR | 7.5
Envoy TOR | 6.5
Scrap TOR | 6.5
Insight TOR | 6.5
JoeDeceives LAT | 6.5
Kremp LAT | 6.5
Ghosty LAT | 7.0
Nastie LAT | 6.5
Snoopy BOS | 7.5
Pentagrxm BOS | 6.0
Priestahh BOS | 6.0
Seany BOS | 6.0

MAP 3 Kills
Huke SEA | 22.5
04 SEA | 22.0

In [34]:
# Convert our list of player props to a dataframe
player_props_df = pd.DataFrame(player_props)
player_props_df

,player,team_abbr,proptype,player_line
0,Huke,SEA,1,23.5
1,04,SEA,1,24.5
2,Breszy,SEA,1,22.0
3,Abuzah,SEA,1,22.0
4,ReeaL,MIA,1,24.0
...,...,...,...,...
67,Nastie,LAT,3,22.5
68,Snoopy,BOS,3,24.0
69,Pentagrxm,BOS,3,21.5
70,Priestahh,BOS,3,23.0


In [17]:
pd.reset_option('display.max_rows')
# pd.set_option('display.max_rows', None)

In [5]:
from webscraper import *

In [19]:
from setup.setup import *

In [6]:
# 1. Get current player props from PrizePicks
player_props_df = scrape_prizepicks()
player_props_df

MAP 1 Kills
MAP 2 Kills
MAP 3 Kills


,player,team_abbr,prop,line
0,Clayster,CAR,1,23.5
1,FeLo,CAR,1,22.5
2,Gwinn,CAR,1,25.5
3,TJHaLy,CAR,1,24.5
4,Lucky,MIA,1,21.5
...,...,...,...,...
67,Huke,SEA,3,22.5
68,CleanX,TOR,3,22.5
69,Envoy,TOR,3,22.5
70,Insight,TOR,3,20.5


In [21]:
# 2. Load cdlDF
cdlDF = load_and_clean_cdl_data()
cdlDF


c:\Users\David Harler Jr\OneDrive\Desktop\dataClass\06-cod-analysis\03 CDL PrizePicks Betting\cdl_prizepicks_betting\cdl-betting-app-v03\setup\setup.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cdlDF = sqlio.read_sql_query("SELECT * FROM cdl_data", conn)


,match_id,match_date,match_day,player,team,map_num,map_name,gamemode,kills,deaths,...,series_result,map_wl,team_abbr,team_icon,opp,opp_abbr,opp_score,total_score,score_diff,dummy_x
0,27193,2023-12-08,Friday,aBeZy,Atlanta FaZe,1,Karachi,Hardpoint,24,24,...,1,W,ATL,FaZe,Boston Breach,BOS,238,488,12,0
1,27193,2023-12-08,Friday,Cellium,Atlanta FaZe,1,Karachi,Hardpoint,21,19,...,1,W,ATL,FaZe,Boston Breach,BOS,238,488,12,0
2,27193,2023-12-08,Friday,Drazah,Atlanta FaZe,1,Karachi,Hardpoint,25,22,...,1,W,ATL,FaZe,Boston Breach,BOS,238,488,12,0
3,27193,2023-12-08,Friday,Simp,Atlanta FaZe,1,Karachi,Hardpoint,33,23,...,1,W,ATL,FaZe,Boston Breach,BOS,238,488,12,0
4,27193,2023-12-08,Friday,Capsidal,Boston Breach,1,Karachi,Hardpoint,24,26,...,0,L,BOS,Breach,Atlanta FaZe,ATL,250,488,-12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4731,27307,2024-05-04,Saturday,Flames,Los Angeles Guerrillas,4,Karachi,Hardpoint,19,24,...,0,L,LAG,Guerrillas,OpTic Texas,TX,250,488,-12,0
4732,27307,2024-05-04,Saturday,Dashy,OpTic Texas,4,Karachi,Hardpoint,19,27,...,1,W,TX,OpTic,Los Angeles Guerrillas,LAG,238,488,12,0
4733,27307,2024-05-04,Saturday,Kenny,OpTic Texas,4,Karachi,Hardpoint,24,26,...,1,W,TX,OpTic,Los Angeles Guerrillas,LAG,238,488,12,0
4734,27307,2024-05-04,Saturday,Pred,OpTic Texas,4,Karachi,Hardpoint,27,24,...,1,W,TX,OpTic,Los Angeles Guerrillas,LAG,238,488,12,0


In [24]:
# 3. Build rosters
rostersDF = build_rosters(cdlDF)
rostersDF

,player,team,team_abbr
0,aBeZy,Atlanta FaZe,ATL
1,Cellium,Atlanta FaZe,ATL
2,Drazah,Atlanta FaZe,ATL
3,Simp,Atlanta FaZe,ATL
4,Beans,Boston Breach,BOS
5,Pentagrxm,Boston Breach,BOS
6,Priestahh,Boston Breach,BOS
7,Seany,Boston Breach,BOS
8,Snoopy,Boston Breach,BOS
9,Clayster,Carolina Royal Ravens,CAR


In [27]:
# 3. Build initial player props 
initial_player_props = build_intial_props(rostersDF)
initial_player_props

,player,team,team_abbr,prop,line
0,aBeZy,Atlanta FaZe,ATL,1,22.0
1,Cellium,Atlanta FaZe,ATL,1,22.0
2,Drazah,Atlanta FaZe,ATL,1,22.0
3,Simp,Atlanta FaZe,ATL,1,22.0
4,Pentagrxm,Boston Breach,BOS,1,22.0
...,...,...,...,...,...
139,Scrap,Toronto Ultra,TOR,3,22.0
140,Dashy,OpTic Texas,TX,3,22.0
141,Kenny,OpTic Texas,TX,3,22.0
142,Pred,OpTic Texas,TX,3,22.0


In [53]:
# 4. Combine initial player props with current player props
updated_player_props = merge_player_props(initial_player_props, player_props_df, rostersDF)
updated_player_props.head(96)

,player,prop,team,team_abbr,line
0,aBeZy,1,Atlanta FaZe,ATL,22.0
1,Cellium,1,Atlanta FaZe,ATL,22.0
2,Drazah,1,Atlanta FaZe,ATL,22.0
3,Simp,1,Atlanta FaZe,ATL,22.0
4,Pentagrxm,1,Boston Breach,BOS,23.0
...,...,...,...,...,...
91,Scrap,2,Toronto Ultra,TOR,6.5
92,Dashy,2,OpTic Texas,TX,6.5
93,Kenny,2,OpTic Texas,TX,6.5
94,Pred,2,OpTic Texas,TX,6.5
